In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import openpyxl

client_id = '7ae275c4289c4961aa84d3944ea547ff'
client_secret = '06cbce79bb0f474aae909c57e6001f45'

client_credentials_manager = SpotifyClientCredentials(client_id= client_id, client_secret= client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# 장르나 유명도 상관 없이 2023년 노래중에서 곡 가져오기
# JSON 형식의 API를 반복을 통해 리스트에 담고, 각 리스트에 담긴 데이터를 PANDAS로 데이터프레임으로 만드는 과정입니다.
artist_name =[]
track_name = []
track_popularity =[]
artist_id =[]
track_id =[]
track_images = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2023', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_images.append(t['album']['images'][0]['url'])
        
track_df = pd.DataFrame({'track_name' : track_name, 'track_id' : track_id, 'track_popularity' : track_popularity, 'artist_name' : artist_name, 'artist_id' : artist_id, 'track_image_link': track_images})

# 데이터프레임을 엑셀 파일로 저장
track_df.to_excel('spotify_tracks_2023.xlsx', index=False)


In [2]:
# 스포티파이 글로벌 일간 차트인 (걸그룹)
blackpink = 'https://open.spotify.com/artist/41MozSoPIsD1dJM0CLPjZF'
twice = 'https://open.spotify.com/artist/7n2Ycct7Beij7Dj7meI4X0'
redvelvet = 'https://open.spotify.com/artist/1z4g3DjTBBZKhvAroFlhOM'
newjeans = 'https://open.spotify.com/artist/6HvZYsbFfjnjFrWF950C9d'
ive = 'https://open.spotify.com/artist/6RHTUrRF63xao58xh9FXYJ'
lesserafim = 'https://open.spotify.com/artist/4SpbR6yFEvexJuaBpgAU5p'
itzy = 'https://open.spotify.com/artist/2KC9Qb60EaY0kW4eH68vr3'
aespa ='https://open.spotify.com/artist/6YVMFz59CuY7ngCxTxjpxE'
gidle = 'https://open.spotify.com/artist/2AfmfGFbe0A0WsTYm0SDTx'
kep1er = 'https://open.spotify.com/artist/5R7AMwDeroq6Ls0COQYpS4'
nmixx = 'https://open.spotify.com/artist/28ot3wh4oNmoFOdVajibBl'

# 스포티파이 글로벌 일간 차트인 (보이그룹)
bts = 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'
straykids = 'https://open.spotify.com/artist/2dIgFjalVxs4ThymZ67YCE'
seventeen = 'https://open.spotify.com/artist/7nqOGRxlXj7N2JYbgNEjYH'
txt = 'https://open.spotify.com/artist/0ghlgldX5Dd6720Q3qFyQB'
enhypen = 'https://open.spotify.com/artist/5t5FqBwTcgKTaWmfEbwQY9'
nctdream = 'https://open.spotify.com/artist/1gBUSTR3TyDdTVFIaQnc02'
exo = 'https://open.spotify.com/artist/3cjEqqelV9zb4BYE3qDQ4O'
treasure = 'https://open.spotify.com/artist/3KonOYiLsU53m4yT7gNotP'
bigbang = 'https://open.spotify.com/artist/4Kxlr1PRlDKEB0ekOCyHgX'

artist_list = [blackpink, twice, redvelvet, newjeans, ive, lesserafim, aespa, gidle, nmixx,
               bts, straykids, seventeen, enhypen, nctdream, exo, bigbang]

In [4]:
artist_list = ["blackpink"]  # 샘플: 아티스트 목록 제한

feature_list = []
seen_tracks = {}  # 트랙 URI를 저장하는 딕셔너리

for artist in artist_list:
    artistprofile = sp.artist(artist_id=artist)
    artistalbums = sp.artist_albums(artist_id=artist, limit=3)  # 샘플: 앨범 제한

    artist_popularity = artistprofile['popularity']

    for album in artistalbums['items']:
        album_uri = album['uri']
        album_tracks = sp.album_tracks(album_uri)

        # 앨범 정보와 트랙 정보 함께 처리
        for track in album_tracks['items']:
            track_number = track['track_number']
            track_uri = track['uri']
            track_id = track['id']
            track_popularity = sp.track(track_id)['popularity']

            if track_uri not in seen_tracks:  # 이미 본 트랙이 아닌 경우
                seen_tracks[track_uri] = True  # 트랙 URI 저장

                try:
                    audiofeatures = sp.audio_features(track_uri)
                    for feature in audiofeatures:
                        feature_list.append([
                            album['artists'][0]['name'],  # artist_name
                            album['artists'][0]['id'],   # artist_id
                            album['artists'][0]['uri'],  # artist_uri
                            artist_popularity,          # artist_popularity
                            album['name'],               # album_name
                            album['id'],                 # album_id
                            album_uri,                   # album_uri
                            album['release_date'],       # release_date
                            # 트랙 정보
                            track_number,                # track_number
                            track['name'] if track['name'] else "",  # track_name
                            track_id,                    # track_id
                            track_uri,                   # track_uri
                            track_popularity,            # track_popularity
                            # 오디오 특징
                            feature['danceability'],     # danceability
                            feature['energy'],          # energy
                            feature['speechiness'],  # speechiness
                            feature['acousticness'],  # acousticness
                            feature['valence'],  # valence
                            feature['duration_ms'],  # duration_ms
                            feature['mode'],  # mode
                            feature['type']  # type (총 28개)
                        ])
                except Exception as e:
                    print(e, track_uri, "skipped")
                    continue

wb = openpyxl.Workbook()
sheet = wb.active
sheet.append([
    "artist_name", "artist_id", "artist_uri", "artist_popularity",
    "album_name", "album_id", "album_uri", "release_date",
    "track_number", "track_name", "track_id", "track_uri", "track_popularity",
    "danceability", "energy", "speechiness", "acousticness",
    "valence", "duration_ms","mode", "type"
])

# 데이터 저장
for data in feature_list:
    sheet.append(data)

# 엑셀 파일 저장
wb.save("kpop_data.xlsx")

print("엑셀 파일 생성 완료: BlackPink_data.xlsx")


In [3]:
artist_list = ["blackpink"]  # 샘플: 아티스트 목록 제한

artist_info_list = []  # 아티스트 정보 저장

for artist in artist_list:
    artistprofile = sp.artist(artist_id=artist)
    artist_popularity = artistprofile['popularity']

    artist_info_list.append([
        artistprofile['name'],
        artistprofile['id'],
        artistprofile['uri'],
        artist_popularity
    ])

    artistalbums = sp.artist_albums(artist_id=artist, limit=3)  # 샘플: 앨범 제한

    album_info_list = []  # 앨범 정보 초기화

    for album in artistalbums['items']:
        album_info_list.append([
            artistprofile['name'],
            artistprofile['id'],
            artistprofile['uri'],
            artist_popularity,
            album['name'],
            album['id'],
            album['uri'],
            album['release_date']
        ])

# 엑셀 파일 저장 (artist_info.xlsx)
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append([
    "artist_name", "artist_id", "artist_uri", "artist_popularity",
    "album_name", "album_id", "album_uri", "release_date"
])

for data in album_info_list:
    sheet.append(data)

wb.save("artist_info.xlsx")
print("엑셀 파일 생성 완료: artist_info.xlsx")


In [26]:
artist_list = [blackpink, twice, redvelvet, newjeans, ive, lesserafim, itzy, aespa, gidle, kep1er, nmixx,
               bts, straykids, seventeen, txt, enhypen, nctdream, exo, treasure, bigbang]

feature_list = []

# 아티스트
for artist in artist_list:
    
    artistprofile = sp.artist(artist_id = artist)
    artistalbums = sp.artist_albums(artist_id = artist, limit = 50)
    
    artist_popularity = artistprofile['popularity']

    # 아티스트별 앨범
    for i in range(len(artistalbums['items'])):
        album_uri = artistalbums['items'][i]['uri']
        album_tracks = sp.album_tracks(album_uri)
        
        # 앨범별 트랙
        for j in range(len(album_tracks['items'])):
            track_number = album_tracks['items'][j]['track_number'] # track_number
            track_uri = album_tracks['items'][j]['uri'] # track uri
            track_id = album_tracks['items'][j]['id'] # track id
            track_popularity = sp.track(track_id)['popularity'] # track popularity (따로 저장)
            audiofeatures = sp.audio_features(track_uri)

            # 트랙별 audio features
            for feature in audiofeatures:
                try:
                    # title 값이 없으면 track_name 값 사용
                    title = track_name if not track_name else track_name

                    # track_name_changed는 생략 (값 제공되지 않음)
                    feature_list.append([
                        artistalbums['items'][0]['artists'][0]['name'],  # artist_name
                        artistalbums['items'][0]['artists'][0]['id'],  # artist_id
                        artistalbums['items'][0]['artists'][0]['uri'],  # artist_uri
                        artist_popularity,  # artist_popularity
                        artistalbums['items'][i]['name'],  # album_name
                        artistalbums['items'][i]['id'],  # album_id
                        album_uri,  # album_uri
                        artistalbums['items'][i]['release_date'],  # release_date
                        track_number,  # track_number
                        title,  # track_name (title 대신 사용)
                        track_id,  # track_id
                        track_uri,  # track_uri
                        track_popularity,  # track_popularity
                        feature['danceability'],  # danceability
                        feature['energy'],  # energy
                        feature['key'],  # key
                        feature['speechiness'],  # speechiness
                        feature['acousticness'],  # acousticness
                        feature['instrumentalness'],  # instrumentalness
                        feature['liveness'],  # liveness
                        feature['valence'],  # valence
                        feature['tempo'],  # tempo
                        feature['duration_ms'],  # duration_ms
                        feature['time_signature'],  # time_signature
                        feature['loudness'],  # loudness
                        feature['mode'],  # mode
                        feature['type']  # type (총 28개)
                    ])
                except Exception as e:
                    print(e, track_uri, "skipped")
                    continue

wb = openpyxl.Workbook()
sheet = wb.active
sheet.append([
    "artist_name", "artist_id", "artist_uri", "artist_popularity",
    "album_name", "album_id", "album_uri", "release_date",
    "track_number", "track_name", "track_id", "track_uri", "track_popularity",
    "danceability", "energy", "key", "speechiness", "acousticness",
    "instrumentalness", "liveness", "valence", "tempo", "duration_ms",
    "time_signature", "loudness", "mode", "type"
])

# 데이터 저장
for data in feature_list:
    sheet.append(data)

# 엑셀 파일 저장
wb.save("kpop_data.xlsx")

print("엑셀 파일 생성 완료: kpop_data.xlsx")


'NoneType' object is not subscriptable spotify:track:0dXjkFzU9Zfyb2k8LwMK36 skipped
'NoneType' object is not subscriptable spotify:track:6BMJWdmLTPOiRZnxygWc3k skipped


In [16]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

client_id = '7ae275c4289c4961aa84d3944ea547ff'
client_secret = '06cbce79bb0f474aae909c57e6001f45'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# JSON 형식의 API를 반복을 통해 리스트에 담고, 각 리스트에 담긴 데이터를 PANDAS로 데이터프레임으로 만드는 과정입니다.
artist_name = []
track_name = []
track_popularity = []
artist_id = []
track_id = []
track_images = []
artist_popularity = []
artist_genres = []
artist_followers = []
artist_country = []
offset = 0

while len(track_name) < 1000:
    track_results = sp.search(q='year:2023', type='track', limit=50, offset=offset, market='KR')
    if len(track_results['tracks']['items']) == 0:
        break
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_images.append(t['album']['images'][0]['url'])
        artist = sp.artist(t['artists'][0]['id'])
        artist_popularity.append(artist['popularity'])
        artist_genres.append(artist['genres'])
        artist_followers.append(artist['followers']['total'])
        artist_country.append('KR')  # 한국(KR)
    offset += 50

track_df = pd.DataFrame({
    'track_name': track_name,
    'track_id': track_id,
    'track_popularity': track_popularity,
    'artist_name': artist_name,
    'artist_id': artist_id,
    'track_image_link': track_images,
    'artist_popularity': artist_popularity,
    'artist_genres': artist_genres,
    'artist_followers': artist_followers,
    'artist_country': artist_country
})
track_df.head()

# 데이터프레임을 엑셀 파일로 저장
track_df.to_excel('spotify_tracks_2023_kpop_kr.xlsx', index=False)
